In [39]:
import os
import glob

import pandas as pd

In [40]:
CITY = 'singapore'
# CITY = 'hanoi'

DATA_DIR = f"../data/{CITY}"

In [41]:
def get_weights(group):
    total = group['Count'].sum()
    group['Weight'] = (group['Count'] / total) * 100.0
    return group

In [43]:
all_dfs = []

csv_files = glob.glob(
    os.path.join(f"{DATA_DIR}/tags", '*.csv')
)
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    all_dfs.append(df)
    
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df = combined_df.groupby('Tag', as_index=False)['Count'].sum()
combined_df[['Key', 'Value']] = combined_df['Tag'].str.extract(r"\('([^']*)', '([^']*)'\)")
combined_df = combined_df.sort_values(by='Count', ascending=False)

combined_df = combined_df[
    combined_df['Key'].isin([
        'surface',
        'highway',
        'lit',
        'sidewalk',
        'footway',
        'foot',
        'traffic_signals',
        'incline',
        'trail_visibility',
    ])
]

combined_df = combined_df.groupby('Key').apply(get_weights)

output_dir = f"{DATA_DIR}/weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
combined_df.to_csv(f"{output_dir}/weights.csv", index=False)